# Devoted Health Tech Assessment

In [2]:
import pandas as pd
import numpy as np
import re
import selenium
from selenium import webdriver
import psycopg2
import configparser
import os
from tqdm import tqdm

### Helper Functions

In [3]:
def get_creds(config_file='creds.ini', section='devoted'):
    """Get credentials for the postgresql database from a file
    """
    config = configparser.ConfigParser()
    config.read(config_file)
    return config[section]


def get_conn():
    """Get a connection to the postgresql database"""
    creds = get_creds()
    try:
        conn = psycopg2.connect(f"dbname={creds['database']} user={creds['user']} host={creds['host']} password={creds['password']}"
                               )
    except:
        print("Couldn't Connect to the Database")
    return conn

    
def query(query, return_query=True, verbose=True):
    """Execute a query on the postgres db
    """
    conn = get_conn()
    cur = conn.cursor()
    if verbose:
        print(query)
    cur.execute(query)
    if return_query:
        colnames = tuple([desc[0] for desc in cur.description])
        df = pd.DataFrame(data=cur.fetchall(),
                         columns=colnames
                         )
        return df

def save_results(df, location="./answers", filename="output.csv"):
    """Save results to csv files"""
    df.to_csv(os.path.join(location, filename), index=False)
    

# SQL Questions

In [4]:
def sql_question(question_query, filename, print_head=True, limit=5):
    """Boilerplate to reduce repeat code"""
    question = query(question_query)
    save_results(question, filename=filename)
    if print_head:
        print(f"First {limit} rows:")
        print(question.head(limit))
        

### Question 1a
How many providers does each group have?

In [5]:
question_1a_query = """select g.medical_group_id,
count(distinct p.full_name) as group_provider_count
from directory.provider_groups as g
inner join directory.providers as p USING (npi)
group by g.medical_group_id
order by group_provider_count desc
"""
sql_question(question_1a_query, 'sql_question_1a.csv')


select g.medical_group_id,
count(distinct p.full_name) as group_provider_count
from directory.provider_groups as g
inner join directory.providers as p USING (npi)
group by g.medical_group_id
order by group_provider_count desc

First 5 rows:
                   medical_group_id  group_provider_count
0  2e03e636383536ed655ca71d0f5d0ca8                    10
1  029fc58de74cc766fe41649e54f39400                    10
2  635e40654456c4c350e35f71e626326d                     8
3  1bb7237aa66907ad613dc4b58e7ffbc2                     8
4  b23188dbe1f0dff437b87a4b87223626                     7


 ### Question 1b
 How many primary care providers?

In [6]:
question_1b_query = """select g.medical_group_id, count(distinct p.full_name) as group_pcp_count
from directory.provider_groups as g
inner join directory.providers as p USING (npi)
where p.is_pcp=true
group by g.medical_group_id
order by group_pcp_count desc
"""
sql_question(question_1b_query, 'sql_question_1b.csv')


select g.medical_group_id, count(distinct p.full_name) as group_pcp_count
from directory.provider_groups as g
inner join directory.providers as p USING (npi)
where p.is_pcp=true
group by g.medical_group_id
order by group_pcp_count desc

First 5 rows:
                   medical_group_id  group_pcp_count
0  2e03e636383536ed655ca71d0f5d0ca8                5
1  029fc58de74cc766fe41649e54f39400                4
2  267bc253248d5924bd68e050948f2741                3
3  1bb7237aa66907ad613dc4b58e7ffbc2                2
4  c6982433a17952c22c86dc5ce58dd87a                2


### Question 2
Which providers do not have associated contact info (order the list by NPI and include the top 20 results)?

In [7]:
question_2_query = """SELECT p.*
FROM   directory.providers as p
LEFT   JOIN directory.provider_contact_info as c USING (npi)
WHERE  c.npi IS null
order by p.npi
limit 20
"""
sql_question(question_2_query, 'sql_question_2.csv')


SELECT p.*
FROM   directory.providers as p
LEFT   JOIN directory.provider_contact_info as c USING (npi)
WHERE  c.npi IS null
order by p.npi
limit 20

First 5 rows:
          npi                   full_name  is_pcp
0  1003868902           MARK ALAN WHITING    True
1  1124069992  ESTEBAN MARTIN KLOOSTERMAN   False
2  1184982993          KERRY-ANN D MILLER    True
3  1225041163             MARISA F. BAKER   False
4  1295936920       DANIEL LAWRENCE COHEN   False


### Quesiton 3a
For each provider, find the record associated with the most recent update (order the list by NPI and include the top 20 results)

In [8]:
question_3a_query = """select a.*
from directory.provider_contact_info a
inner join (
select distinct npi,
first_value(update_date) over (partition by npi order by update_date desc) as last
from directory.provider_contact_info
) b
on a.npi = b.npi and a.update_date = b.last
order by a.npi
limit 20
"""
sql_question(question_3a_query, 'sql_question_3a.csv')


select distinct a.*
from directory.provider_contact_info a
inner join (
select distinct npi,
first_value(update_date) over (partition by npi order by update_date desc) as last
from directory.provider_contact_info
) b
on a.update_date = b.last
order by a.npi
limit 20

First 5 rows:
          npi                  address1 address2        city state    zip  \
0  1003872375  8391 W Oakland Park Blvd              Sunrise    FL  33351   
1  1013183425           6900 SW 80th St                Miami    FL  33143   
2  1013183425           6900 SW 80th St                Miami    FL  33143   
3  1013183425           6900 SW 80th St                Miami    FL  33143   
4  1023038494           3450 Lantana Rd           Lake Worth    FL  33462   

        phone         fax data_source  confidence_score update_date  
0  9547492184                       F          0.016811  2018-07-22  
1  7864675701  7866624649           C          0.972215  2018-06-27  
2  7866628118  7866624649           B        

### Question 3b
For each provider, find the record associated with the highest quality source that has been updated in the last 60 days (order the list by NPI and include the top 20 results)

**ISSUE:** the database doesn't have data within the last 60 days so I changed it to 90 days

In [9]:
n_days = 90
question_3b_query = f"""select a.*
from directory.provider_contact_info a
inner join
(
select npi, update_date,
first_value(confidence_score) over (partition by npi order by confidence_score desc) as best_score
from directory.provider_contact_info
where update_date >= current_date - {n_days}
) b
on a.npi = b.npi and a.update_date = b.update_date and a.confidence_score = b.best_score
"""
sql_question(question_3b_query, 'sql_question_3b.csv')

select a.*
from directory.provider_contact_info a
inner join
(
select npi, update_date,
first_value(confidence_score) over (partition by npi order by confidence_score desc) as best_score
from directory.provider_contact_info
where update_date >= current_date - 90
) b
on a.npi = b.npi and a.update_date = b.update_date and a.confidence_score = b.best_score

First 5 rows:
          npi                   address1 address2         city state    zip  \
0  1013183425            6900 SW 80th St                 Miami    FL  33143   
1  1699838979  1100 Goodlette Road North                Naples    FL  34102   
2  1356430623   38035 Medical Center Ave           Zephyrhills    FL  33540   
3  1659338457        20803 Biscayne Blvd              Aventura    FL  33180   
4  1447797691   1600 Lakeland Hills Blvd              Lakeland    FL  33805   

        phone         fax data_source  confidence_score update_date  
0  7866628531  7866624649           C          0.157208  2018-08-14  
1  2394340656 

### Question 3c
For each provider/data source combination, find the current phone number, previous phone number, and flag whether the number has changed (order the list by provider then data source and include the top 20 results)

In [10]:
question_3c_query = """
select a.*
from 
(
select *,
lag(phone, 1)
over (partition by npi, data_source order by update_date) = lag(phone, 0, null)
over (partition by npi, data_source order by update_date, null) as is_phone_same
from directory.provider_contact_info
order by npi, data_source
) a
where a.is_phone_same = false
order by npi, data_source
limit 20
"""
sql_question(question_3c_query, 'sql_question_3c.csv')



select a.*
from 
(
select *,
lag(phone, 1)
over (partition by npi, data_source order by update_date) = lag(phone, 0, null)
over (partition by npi, data_source order by update_date, null) as is_phone_same
from directory.provider_contact_info
order by npi, data_source
) a
where a.is_phone_same = false
order by npi, data_source
limit 20

First 5 rows:
          npi                  address1 address2            city state    zip  \
0  1003872375  8391 W Oakland Park Blvd                  Sunrise    FL  33351   
1  1013183425           6900 SW 80th St                    Miami    FL  33143   
2  1023440989       293 NW Peacock Blvd           Port St. Lucie    FL  34986   
3  1023440989       293 NW Peacock Blvd           Port St. Lucie    FL  34986   
4  1023440989       293 NW Peacock Blvd           Port St. Lucie    FL  34986   

        phone         fax data_source  confidence_score update_date  \
0  9547492184                       F          0.016811  2018-07-22   
1  7866628531  7866

# Data Cleaning

In [11]:
data_location = './data/new_phone_records.csv'
data = pd.read_csv(data_location)


### Question
Using python, clean these phone numbers to prepare them for upload into the database as a 10-digit string. Attach your code for cleaning the data as well as a CSV with your resulting file for upload sorted by NPI then phone number.

In [12]:
PHONE_REGEX = re.compile(r'x[0-9][0-9]?|ext.\s[0-9][0-9]?|^\+[0-9]|^[0-9]-|-|\s|\.|\(|\)')
def clean_phone_numbers(dirty_number):
    clean_number = re.sub(PHONE_REGEX, '', dirty_number)
    clean_number = clean_number.replace('\x07', '').replace('\x01', '').replace('\x08', '')
    return clean_number
    

In [13]:
data['original_phone'] = data.phone
data['phone'] = data.phone.apply(clean_phone_numbers)
save_results(data.sort_values(by=['npi', 'phone']), filename='data_cleaning.csv')


#### Prepare to upload the data
Prepare the data to be uploaded as a table to the database (see commented out code below)

I wasn't sure if you were looking for a table create query and/or actually wanted the data to be uploaded, but I wrote out a potential method using pandas

In [14]:
# data.to_sql('directory.mallen_clean_phone', con=get_conn(), if_exists='replace')


### Comments
- Some of the number are don't have enough numbers in them (i.e. 9 numbers istead of 10) while others have too many numbers (see below)
  - This could be an entry error
- Some numbers are malformatted and have characters like `\x073`, `\x01` or `\x08`
  - I manually dealt with those values


In [15]:
data.phone.str.len().value_counts()


10    105
9       2
11      1
Name: phone, dtype: int64

### Compare

#### (a)
How many providers who did not previously have a phone number in the database have one in the new data source?


In [16]:
missing_numbers_query = """select count(*)
from directory.provider_contact_info
where phone = ''
"""

print(f"""There are {query(missing_numbers_query, verbose=False).values[0][0]} providers
      who had no phone number listed prior to the new data""")


There are 0 providers
      who had no phone number listed prior to the new data


**Note:** There doesn't seem to be any provider in the dataset that was missing a phone number

#### (b) 
How many of the records in the new data source match a phone number we already had associated with that provider


In [17]:
contact_phone_data = query("""select npi, phone, update_date
                           from directory.provider_contact_info""",
                          verbose=False)


In [18]:
recent_contact_phone = (contact_phone_data.
                        sort_values(by='update_date').
                        groupby('npi').
                        agg(lambda x: x.iloc[0]).
                        reset_index()
                       )
recent_contact_phone['npi'] = pd.to_numeric(recent_contact_phone.npi)


In [19]:
print(f"""
There are
{(pd.merge(recent_contact_phone, data, on=['npi', 'phone'], how='inner')
).shape[0]}
phone numbers in the new dataset that had already been updated in the contact list"""
     )
     


There are
38
phone numbers in the new dataset that had already been updated in the contact list


#### (c)
How many records in the new data source conflict with a phone number we already had associated with that provider?


In [20]:
merged_phones = pd.merge(recent_contact_phone, data, on='npi', how='inner')


In [21]:
print(f"""
There are
{merged_phones[merged_phones.phone_x != merged_phones.phone_y].shape[0]}
phone numbers in the new dataset that conflict with the number in the contact list"""
     )



There are
63
phone numbers in the new dataset that conflict with the number in the contact list


# Scraping

### Question 1
Using python, scrape the ratings from this site and export them to a CSV for upload to the database. Attach your code for scraping the ratings as well as the data in a CSV sorted by NPI.

In [101]:
driver = webdriver.Chrome("/Users/brentonmallen/chromedriver")
pages = range(1,4)
table_xpath = "/html/body/center[2]/font/table/tbody"


In [102]:
scraped_ratings = []
for page in tqdm(pages):
    url = f"http://ec2-54-90-234-76.compute-1.amazonaws.com/docstars_pg{page}.html"
    driver.get(url);
    rating_table = driver.find_element_by_xpath(table_xpath)
    rows = rating_table.find_elements_by_xpath('tr')
    for i, r in enumerate(rows):
        if i == 0 and page==1:
            headers = [h.text.lower() for h in r.find_elements_by_tag_name('th')]
            scraped_ratings.append(headers)
        else:
            row_data = [col.text for col in r.find_elements_by_tag_name('td')]
            scraped_ratings.append(row_data)
            

100%|██████████| 3/3 [00:10<00:00,  3.63s/it]


In [104]:
scraped_data = pd.DataFrame(data=scraped_ratings[1:], columns=scraped_ratings[0])
save_results(scraped_data.sort_values(by='npi'), filename='scraped_ratings.csv')


### Question 2
What is the average rating for providers in the plan's network (that is, the providers in the SQL database you looked at above)? What about for the plan's Primary Care providers? (you can do this either in SQL or python, but please include your code along with your answer)

In [108]:
network_providers = query('select * from directory.providers', verbose=False)


In [120]:
ratings_intersection = pd.merge(network_providers, scraped_data, on=['npi'], how='inner')
                      
provider_avg_rating = (ratings_intersection.
                       rating.
                       astype(float).
                       mean().
                       round(3)
                      )

pcp_avg_rating = (ratings_intersection[ratings_intersection.is_pcp == True].
                       rating.
                       astype(float).
                       mean().
                       round(3)
                      )


In [118]:
print(f"The average DJ's Docstars rating for in network providers is: {provider_avg_rating}")


The average DJ's Docstars rating for in network providers is: 3.283


In [121]:
print(f"The average DJ's Docstars rating for in network PCPs is: {pcp_avg_rating}")


The average DJ's Docstars rating for in network PCPs is: 3.506
